# Analysis of the Mutuals Funds from Schwab

### Author: Jefferson Bien-Aimé
### Date : December 8th, 2025

## Executive Summary

## Table of Contents

1. Introduction
2. Problem Statement / Research Question
3. Data Description
4. Setup and Environment
5. Data Loading
6. Data Preparation
7. Model Planning
8. Model Building / Analysis
9. Discussion & Interpretation
10. Conclusion
11. References
12. Appendix

In [ ]:
testing

## Introduction

## Problem Statement / Research Question

## Data Description

## Setup and Environment

In [ ]:
!pip install google-cloud-storage

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os
from google.cloud import storage
from io import BytesIO
import requests

## Data Loading

### Functions

In [ ]:
def list_public_bucket(bucket_name, course, project):
    prefix = f"{course}/Project {project}/"

    client = storage.Client.create_anonymous_client()
    bucket = client.bucket(bucket_name)
    blobs = client.list_blobs(bucket, prefix=prefix)
    print(f"Listing files in public bucket '{bucket_name}' under '{prefix}':")
    filenames = []
    for blob in blobs:
        print(blob.name)
        filenames.append(blob.name)

    return filenames

In [ ]:
from google.cloud import storage
import pandas as pd
from io import BytesIO
import os
import json

def gcs_file_to_dataframe(bucket_name: str, blob_path: str):

    # Anonymous client for public buckets
    client = storage.Client.create_anonymous_client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_path)

    # Download file content as bytes
    data = blob.download_as_bytes()

    # Detect file extension
    _, ext = os.path.splitext(blob_path.lower())

    if ext == ".csv":
        df = pd.read_csv(BytesIO(data))

    elif ext in [".xls", ".xlsx"]:
        df = pd.read_excel(BytesIO(data))

    elif ext == ".parquet":
        df = pd.read_parquet(BytesIO(data))

    elif ext == ".json":
        # Convert bytes → Python object
        text = data.decode("utf-8")
        json_obj = json.loads(text)

        # If JSON is a list → NORMAL DataFrame
        if isinstance(json_obj, list):
            df = pd.DataFrame(json_obj)

        # If JSON is a dict → flatten into rows
        elif isinstance(json_obj, dict):
            df = pd.json_normalize(json_obj)

        else:
            raise ValueError("Unsupported JSON structure")

    else:
        raise ValueError(f"Unsupported file type: {ext}")

    return df

In [ ]:
# Example
BUCKET_NAME = "msba-online-data"
PROJECT_NUMBER = "05"
COURSE = "CIS9650"
salaries_raw = list_public_bucket(BUCKET_NAME,COURSE,PROJECT_NUMBER)

Listing files in public bucket 'msba-online-data' under 'CIS9650/Project 05/':
CIS9650/Project 05/
CIS9650/Project 05/response_0023e4c1-b77e-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_018aa811-b77b-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_019321a1-b77d-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_01d12a21-b77f-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_01df9b51-b77a-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_02546b41-b77c-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_0341eda1-b77e-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_04a44421-b77b-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_04a606f1-b77d-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_04f3c6e1-b77f-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_04fcb9d1-b77a-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_05711491-b77c-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_0661a431-b77e-11f0-a457-2128cc60a

In [ ]:
df_raw = pd.DataFrame()
for element in salaries_raw[1:]:
    df = gcs_file_to_dataframe(BUCKET_NAME,  element)
    df_raw = pd.concat([df_raw, df])
print(df_raw.shape)

   total  hidden                                               rows
0    250       0  [{'uuid': '406bc2ba-a769-45f9-bbbb-73af0bfef9f...


In [ ]:
df_expanded = df.explode("rows", ignore_index=True)

# Convert the dict into columns
detail_cols = df_expanded["rows"].apply(pd.Series)

# Merge back and drop the dict column
df_flat = pd.concat([df_expanded.drop(columns=["rows"]), detail_cols], axis=1)

df_flat

,total,hidden,uuid,company,title,jobFamily,level,focusTag,yearsOfExperience,yearsAtCompany,...,totalOutstandingShares,latestCompanyValuation,companySize,fundingStage,annualTargetBonusValue,annualTargetBonusPercentage,additionalBonuses,anonymise,hidden,locationSlug
0,250,0,406bc2ba-a769-45f9-bbbb-73af0bfef9f6,PwC,Auditor,Accountant,L2,Audit,2,2,...,None,None,None,None,NaN,NaN,None,False,False,NaN
1,250,0,c4800146-d1f1-4760-b0f4-0716c2e3e5a5,The Wonderful Company,Tax Accountant,Accountant,L2,Tax,5,2,...,None,None,None,None,NaN,NaN,None,False,False,NaN
2,250,0,5ffef2f4-ee14-4ec7-87bd-28d8b3576311,Deloitte Advisory,Audit,Accountant,Analyst 1,Audit,1,1,...,None,None,None,None,NaN,NaN,None,False,False,NaN
3,250,0,c04f77fb-50e3-4b95-8bf1-5e34837f1f06,Hepa Graff Professional Cleaning,Accountant,Accountant,None,General Ledger,3,3,...,None,None,None,None,NaN,NaN,None,False,False,NaN
4,250,0,99761398-24f9-4c0a-93cb-302d0a677d0e,KPMG,Auditor,Accountant,Associate Accountant,Audit,0,0,...,None,None,None,None,NaN,NaN,None,False,False,NaN
5,250,0,0b39be43-76ac-4930-859d-07314d8bc6c5,Aldi,Accountant,Accountant,L1,General,2,2,...,None,None,None,None,NaN,NaN,None,False,False,NaN
6,250,0,79973511-be9b-473a-ac92-b31a9906a232,Sea,Accountant,Accountant,None,General Ledger,6,0,...,None,None,None,None,NaN,NaN,None,False,False,NaN
7,250,0,08daf83cd87bfcf0fc38a794c97c13c5e094b9df489719...,False,False,Accountant,False,False,5-10,2-4,...,None,None,None,None,NaN,NaN,None,True,False,NaN
8,250,0,eaf48510-9a9b-4b06-a33a-1c5bde02a5cf,Ernst and Young,Accountant,Accountant,Staff 2,Audit,2,2,...,None,None,None,None,NaN,NaN,None,False,False,NaN
9,250,0,5547ff88-6f3b-493f-9382-4db92246ab1a,Bayer,Accountant,Accountant,VS 1.3,Treasury,30,26,...,None,None,None,None,NaN,NaN,None,False,False,NaN


## Data Preparation

In [ ]:
# Check duplicates
# Get Unique Values on each column
# NULL
# Outliers

## Model Planning

## Model building / Analysis

## Discussion and Results

## Conclusion

## References

## Appendix